<img src="img/csdms_logo.jpg">

# Visualization with `ipywidgets`

Let's visualize the evolution of the 2D temperature field as heat diffuses across the plate. We can do this interactively with the `ipywidgets` package.

**Note:** Depending on the current state of your Python installation, you may have to install the `ipywidgets` package, have `notebook>=4.2.0`, and enable widgets.

```bash
$ pip install ipywidgets
$ jupyter nbextension enable --py --sys-prefix widgetsnbextension
```

Start with the necessary imports.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from ipywidgets import interact
from bmi_live.bmi_diffusion import BmiDiffusion

Define a function that runs the `Diffusion` model through its BMI for a fixed number of time steps. Store the temperture field from each time step in a list. Return the list.

In [ ]:
def run_model(n_steps=10):
    model = BmiDiffusion()
    model.initialize('../data/diffusion.yaml')
    grid_shape = model.get_grid_shape(0)
    
    initial_temp = model.get_value('plate_surface__temperature')
    initial_temp[20] = 100.
    model.set_value('plate_surface__temperature', initial_temp)
    
    temp = [initial_temp.reshape(grid_shape)]
    for _ in range(n_steps):
        model.update()
        new_temp = model.get_value('plate_surface__temperature')
        temp.append(new_temp.reshape(grid_shape))
    
    model.finalize()
    return temp

Run the model for awhile.

In [ ]:
temperature = run_model(50)

Define a function to visualize the temperature field at a time step (defined by an index into the list of temperature fields).

In [ ]:
xmin, xmax = 0., temperature[0].shape[1]
ymin, ymax = 0., temperature[0].shape[0]
im = plt.imshow(temperature[0], vmax=100, extent=[xmin, xmax, ymin, ymax], origin='upper', cmap='gist_stern')
plt.title('Plate Temperature')
plt.colorbar().ax.set_ylabel('Temperature (K)')

In [ ]:
def plot_temperature(time_index=0):
    im.set_data(temperature[time_index])

Make an interactive plot of temperature over time.

In [ ]:
interact(plot_temperature, time_index=(0, 50, 1))